In [1]:
!kaggle datasets download -d jp797498e/twitter-entity-sentiment-analysis --unzip

Dataset URL: https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis
License(s): CC0-1.0
  0% 0.00/1.99M [00:00<?, ?B/s]
100% 1.99M/1.99M [00:00<00:00, 119MB/s]


In [2]:
import pandas as pd
training_data = pd.read_csv("twitter_training.csv")

In [3]:
# what is the data liek ?
training_data
# splitting into valadation data
validation_data=pd.read_csv("twitter_validation.csv")
validation_data

,3364,Facebook,Irrelevant,"I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣"
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,Neutral,Now the President is slapping Americans in the...
4,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...
...,...,...,...,...
994,4891,GrandTheftAuto(GTA),Irrelevant,⭐️ Toronto is the arts and culture capital of ...
995,4359,CS-GO,Irrelevant,tHIS IS ACTUALLY A GOOD MOVE TOT BRING MORE VI...
996,2652,Borderlands,Positive,Today sucked so it’s time to drink wine n play...
997,8069,Microsoft,Positive,Bought a fraction of Microsoft today. Small wins.


In [4]:

#lets chnage the colum names
training_data.columns=['SerialNo','Company','labels','text']
validation_data.columns=['SerialNo','Company','labels','text']

In [5]:
#lets see how many null vals are there
print(training_data.isnull().sum())
# now the same for valadation data
validation_data.isnull().sum()

SerialNo      0
Company       0
labels        0
text        686
dtype: int64


SerialNo    0
Company     0
labels      0
text        0
dtype: int64

In [6]:
training_data.duplicated().sum()
validation_data.duplicated().sum()
sentiment=pd.concat([training_data,validation_data], ignore_index=True)

In [7]:
sentiment.head(10)

,SerialNo,Company,labels,text
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
5,2402,Borderlands,Positive,So I spent a few hours making something for fu...
6,2402,Borderlands,Positive,So I spent a couple of hours doing something f...
7,2402,Borderlands,Positive,So I spent a few hours doing something for fun...
8,2402,Borderlands,Positive,So I spent a few hours making something for fu...
9,2402,Borderlands,Positive,2010 So I spent a few hours making something f...


In [8]:
sentiment.dropna(inplace=True)
sentiment.drop_duplicates(inplace=True)
sentiment.shape

(72138, 4)

In [9]:
# now lests look at the data once more ....
print(sentiment.head(10))
# now lets drop siralnumber adn company
sentiment.drop(['SerialNo','Company'],axis=1,inplace=True)

   SerialNo      Company    labels  \
0      2401  Borderlands  Positive   
1      2401  Borderlands  Positive   
2      2401  Borderlands  Positive   
3      2401  Borderlands  Positive   
4      2401  Borderlands  Positive   
5      2402  Borderlands  Positive   
6      2402  Borderlands  Positive   
7      2402  Borderlands  Positive   
8      2402  Borderlands  Positive   
9      2402  Borderlands  Positive   

                                                text  
0  I am coming to the borders and I will kill you...  
1  im getting on borderlands and i will kill you ...  
2  im coming on borderlands and i will murder you...  
3  im getting on borderlands 2 and i will murder ...  
4  im getting into borderlands and i can murder y...  
5  So I spent a few hours making something for fu...  
6  So I spent a couple of hours doing something f...  
7  So I spent a few hours doing something for fun...  
8  So I spent a few hours making something for fu...  
9  2010 So I spent a few hours 

In [10]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
#lets add re
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [11]:
# lets define a funtion to clean the data
def cleaner(text):
    test = re.sub(r'@[A-Za-z0-9]+', '', text) #Removing @mentions
    test = re.sub(r'#', '', text) # Removing '#' hash tag
    test = re.sub(r'RT[\s]+', '', text) # Removing RT
    test = re.sub(r'https?:\/\/\S+', '', text) # Removing the hyper link
    #removing specal char
    test = re.sub(r'[^\w\s]', '', text)
    #removing single char
    test = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    #remove any chat that is not alphabetical
    test = re.sub(r'[^a-zA-Z]', '', text)
    #remove emohies
    test = re.sub(r'[^\x00-\x7F]+', '', text)

    # remove words less than 3 char
    text = re.sub(r'\b\w{1,3}\b', '', text)

    # now lower case
    text = text.lower()

    #remove numbers
    text = re.sub(r'[0-9]', '', text)

    # now lets tokenize
    text = word_tokenize(text)

    lei = WordNetLemmatizer()
    # now lets lemmatize

    text = [lei.lemmatize(word) for word in text if not word in set(stopwords.words('english'))]
    #now stop words
    text = [word for word in text if not word in set(stopwords.words('english'))]

    #remove duplicates
    text = list(set(text))
    cleaned_text = ' '.join(text)

    return cleaned_text

In [12]:
x = sentiment.drop('labels', axis=1)
y = sentiment.labels

In [13]:
texts = list(x['text'])

In [15]:
cleaned_text = [cleaner(texts) for texts in texts]

In [18]:
# now lets split the data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(cleaned_text, y, test_size=0.2, random_state=42)

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_vocab = 30000
tokenizer = Tokenizer(num_words=max_vocab)
tokenizer.fit_on_texts(x_train)
word_idx = tokenizer.word_index  # accessing word index... the coe above def didnt take like 10 mins to ruN!!
v = len(word_idx)
print("the size of vocab =", v)  # Corrected spacing
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

the size of vocab = 25545


In [19]:
x_train = pad_sequences(x_train, maxlen=100)
x_test = pad_sequences(x_test, maxlen=100)

In [20]:
#Lets look at x_train
x_train[0:5]


array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          61,  456, 1156, 2110,  181,  117,   11, 1794,   49, 1231,    3,
        3343, 1746,   54,  108,  301,    4,  289,  755, 6256,  773,  260,
         516],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    

In [21]:
# now we start with the mind..... aka tensorflow.. let simpliment a LSTM model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
import tensorflow as tf
model = Sequential()
model.add(Embedding(max_vocab, 100, input_length=100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          3000000   
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 3117377 (11.89 MB)
Trainable params: 3117377 (11.89 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
#now let train with 10 ephos
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))


Epoch 1/10


UnimplementedError: Graph execution error:

Detected at node binary_crossentropy/Cast defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/python